In [12]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl                                           # for interactive map visualization
from dash import dcc, html                                          # components and HTML elements
import plotly.express as px                                         # create interactive charts
from dash import dash_table                                         # for interactive data tables
from dash.dependencies import Input, Output, State                  # callback interactions
import base64                                                       # encoding logo image
JupyterDash.infer_jupyter_proxy_config()                            

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np                                                  
import pandas as pd                                                    
import matplotlib.pyplot as plt


# import custom CRUD Python module for MongoDB operations
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# database connection credentials and settings
username = "aacuser"
password = "Ohmongosh"
host = "127.0.0.1"
port = 27017
database = "aac"
collection = "animals"

# Connect to database via CRUD Module
db = AnimalShelter(username, password, host, port, database, collection)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
# initialize dash app
app = JupyterDash(__name__)

# load and encode Grazioso Salvare's logo for displace
image_filename = 'GraziosoSalvareLogo.png' 

# convert image to base64 string for embedding in HTML
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


# define the dashboard layout structure
app.layout = html.Div([
    # header section with logo and title
    html.Div([
        # logo container
        html.Div([
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'height': '100px', 'margin-right': '20px'})
        ], style={'display': 'inline-block', 'vertical-align': 'middle'}),
    
        # title and identifier container
        html.Div([
            html.Center(html.B(html.H1('Grazioso Salvare Animal Shelter Dashboard'))),
            html.P('Created by: Shimin Chan', style={'font-style': 'italic', 'color': '#666'})
        ], style={'display': 'inline-block', 'vertical-align': 'middle'})
    ], style={'text-align': 'center', 'padding': '20px', 'background-color': '#f0f0f0'}),
        
    html.Hr(),
    
    # interactive filer section with radio buttons
    html.Div([
        html.Label('Filter by Rescue Type:', style={'font-weight': 'bold', 'font-size': '18px'}),
        html.Br(),
        
        # radio buttons for selecting rescue type filters
        dcc.RadioItems(
            id='filter-type',              # componenet ID for callback reference
            options=[
                {'label': 'Water Rescue', 'value': 'Water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
                {'label': 'Reset (Show All)', 'value': 'Reset'}
            ],
            # default selection
            value='Reset',
            labelStyle={'display': 'inline-block', 'margin-right': '20px'},
            style={'font-size': '16px'}
        )
    ], style={'padding': '20px', 'background-color': '#f9f9f9', 'border-radius': '5px'}),
    
    html.Hr(),
    
# data table displaying animal shelter data    
    dash_table.DataTable(id='datatable-id',                    # component ID for callback reference                   
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],          # create columns from DataFrame column names
                         data=df.to_dict('records'),
                         editable = False,                     # enable editing
                         filter_action = "native",             # enable filtering by column
                         sort_action = "native",               # enable sorting by column
                         sort_mode = "multi", 
                         column_selectable = False,            # set column cannot be selected
                         row_selectable = "single",            # enable row selection (REQUIRED - must be 'single' for map)
                         selected_rows = [0],                     # select first row by default
                         page_action = "native",               # enable pagination
                         page_current = 0,
                         page_size = 10,                       # show 10 rows per page
                         # table container set to scrollable
                         style_table = {'overflowX' : 'auto',
                               'overflowY' : 'auto',
                               'maxHeight' : '600px'
                          },
                         # styling for individual cell
                         style_cell = {
                                'textAlign' : 'left',
                                'minWidth' : '100px', 'width' : '150px', 'maxWidth' : '180px',
                                'whiteSpace' : 'normal'
                         },
                         # styling for header row
                         style_header = {
                                'backgroundColor' : 'rgb(230, 230, 230)',
                                'fontWeight' : 'bold',
                                'textAlign' : 'center'
                        },
                         # styling for alternating rows and selected state
                         style_data_conditional = [
                                {
                                    'if' : {'row_index' : 'odd'},
                                    'backgroundColor' : 'rgb(248, 248, 248)'
                                },
                                {
                                    'if' : {'state' : 'selected'},
                                    'backgroundColor' : 'rgb(200, 220, 240)',
                                    'border' : '1px solid blue'
                                }
                            ]
                        ),
                         
    html.Br(),
    html.Hr(),
    
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        # pie chart container         
        html.Div(
            id='graph-id',
            className='col s12 m6',
            style={'width': '50%', 'padding': '10px'}
            ),
        # geolocation map container
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={'width': '50%', 'padding': '10px'}
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
# callback to update data table based on selected rescue type fulter
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])                 # triggered by radio button selection

# filter the data table based on rescure type selection
def update_dashboard(filter_type):
    
    # filter for Water Resecue
    if filter_type == 'Water':
        query = {
            '$and': [
                {'$or': [
                    {'breed': 'Labrador Retriever Mix'},
                    {'breed': 'Chesapeake Bay Retriever'},
                    {'breed': 'Newfoundland'}
                ]},
                {'sex_upon_outcome': 'Intact Female'},
                {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
            ]
        }
    
    # filter for Mountain or Wilderness Rescue
    elif filter_type == 'Mountain':
        query = {
            '$and': [
                {'$or': [
                    {'breed': 'German Shepherd'},
                    {'breed': 'Alaskan Malamute'},
                    {'breed': 'Old English Sheepdog'},
                    {'breed': 'Siberian Husky'},
                    {'breed': 'Rottweiler'},
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}}
            ]
        }
        
    # filter for Disaster or Individual Tracking
    elif filter_type == 'Disaster':
        query = {
            '$and': [
                {'$or': [
                    {'breed': 'Doberman Pinscher'},
                    {'breed': 'German Shepherd'},
                    {'breed': 'Golden Retriever'},
                    {'breed': 'Bloodhound'},
                    {'breed': 'Rottweiler'}
                ]},
                {'sex_upon_outcome': 'Intact Male'},
                {'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}}
            ]
        }
    # reset filter    
    else:
        query = {}
     
    # query database using CRUD module and convert results to DataFrame
    df_filtered = pd.DataFrame.from_records(db.read(query)) 
    
    # remove '_id' column if present to prevent data table errors and better view
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)        
        
    # return filtered data as dictionary for table display
    return df_filtered.to_dict('records')

# callback to update pir chart based on the fitlered data
@app.callback(
    Output('graph-id', "children"),                              # update pie chart container
    [Input('datatable-id', "derived_virtual_data")]              # triggered by table data changes
)

# generate pir chart showing breed distribution for filtered data
def update_graphs(viewData):
    
    # handle case where no data is available
    if viewData is None:
        return html.Div("No data available")
    
    # convert table data dictionary to DataFrame
    dff = pd.DataFrame.from_dict(viewData)
    
    # handle case where filtered data is empty
    if dff.empty:
        return html.Div("No data to display")
    
    # count occurrences of each breed and rest index to create DataFrame
    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']
    
    # limit to the first 10 breeds to display
    breed_counts = breed_counts.head(10)
    
    # create and return pir chart using plotly express
    return [dcc.Graph(figure=px.pie(breed_counts, values='count', names='breed', title='Top 10 Breed Distributor', hole=0.3))]


#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),                   # update table styling
    [Input('datatable-id', 'selected_columns')]                         # triggered by column selection
)

# apply conditional styling to highlight selected columns
def update_styles(selected_columns):


    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in (selected_columns or [])]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):  
    if viewData is None:
        return html.Div("No data available for map.")
    
    if not viewData:
        return html.Div("No animals match the selected filter.")
    
    dff = pd.DataFrame.from_dict(viewData)
    
    
    if index is None or len(index) == 0:
        row = 0
    else: 
        row = index[0]
        
    if row >= len(dff):
        row = 0
        
    # create and return interactive map centered on Austin,TX
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            
            # marker for selected animal's location, position from columns 13 - latutude and columns 14 - longitude
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                
                # tooltip shows breed on hover
                dl.Tooltip(dff.iloc[row,4]),
                
                # popup show detailed information on click
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9]),             # name
                    html.H1("Breed"),
                    html.P(dff.iloc[row, 4]),            # breed
                    html.H1("Age"),
                    html.P(dff.iloc[row, 15])            # age
                ])
            ]) 
        ])
    ]


# open dashboard in new browser tab, specifies 8050 as the server port, enable developer tools, enable automatic regresh on code changes
app.run_server(mode='external', port=8050, dev_tools_ui=True, dev_tools_hot_reload=True) 

You are connected to MongoDB successfully .
Dash app running on https://kineticstudio-lionrapid-3000.codio.io/proxy/8050/
